In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import sys
sys.path.append('../src')

# 커스텀 함수 import
from prediction_utils import (create_lag_features, create_ai_features, 
                             create_interaction_features, build_ensemble_model, 
                             predict_scenarios)

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

print("🚀 통합 예측 모델 구축 시작!")


In [ ]:
# 전처리된 데이터 로딩
try:
    export_data = pd.read_csv('../output/export_data_processed.csv', encoding='cp949')
    claims_data = pd.read_csv('../output/claims_data_processed.csv', encoding='cp949')
    risk_data = pd.read_csv('../output/risk_data_processed.csv', encoding='cp949')
    
    # 년월 컬럼을 datetime으로 변환
    export_data['년월'] = pd.to_datetime(export_data['년월'])
    
    print("✅ 데이터 로딩 완료")
    print(f"수출 데이터: {export_data.shape}")
    print(f"보상 데이터: {claims_data.shape}")
    print(f"위험지수 데이터: {risk_data.shape}")
    
except Exception as e:
    print(f"❌ 데이터 로딩 실패: {e}")
    print("먼저 01_data_preparation.ipynb를 실행해주세요!")


In [ ]:
# 1-1. 시차 기반 피처 생성
print("🔄 시차 기반 피처 생성")
lag_features = create_lag_features(export_data)

if not lag_features.empty:
    print(f"✅ 시차 피처 생성 완료: {lag_features.shape}")
    print("\n📊 시차 피처 샘플:")
    print(lag_features.head())
    
    # 주요 통계
    print(f"\n📈 주요 통계:")
    print(f"- 평균 총수출액: {lag_features['총수출액'].mean():.0f}")
    print(f"- 평균 수출변동성: {lag_features['수출변동성'].mean():.0f}")
    print(f"- 평균 계절성 변동계수: {lag_features['계절성_변동계수'].mean():.4f}")
else:
    print("❌ 시차 피처 생성 실패")

# 1-2. AI 위험지수 기반 피처 생성
print("\n🤖 AI 위험지수 기반 피처 생성")
ai_features = create_ai_features(risk_data)

if not ai_features.empty:
    print(f"✅ AI 피처 생성 완료: {ai_features.shape}")
    print("\n📊 AI 피처 샘플:")
    print(ai_features.head())
    
    # 위험등급별 분포
    print(f"\n🎯 위험등급별 분포:")
    print(ai_features['위험등급'].value_counts())
else:
    print("❌ AI 피처 생성 실패")


In [ ]:
# 1-3. 상호작용 피처 생성 및 타겟 변수 준비
print("⚡ 상호작용 피처 생성")

if not lag_features.empty and not ai_features.empty:
    # 상호작용 피처 생성
    integrated_features = create_interaction_features(lag_features, ai_features)
    
    if not integrated_features.empty:
        print(f"✅ 통합 피처 생성 완료: {integrated_features.shape}")
        print(f"📊 총 피처 수: {len(integrated_features.columns)}")
        
        # 보상 데이터와 병합하여 타겟 변수 생성
        claims_agg = claims_data.groupby('국가명').agg({
            '보상금': 'sum',
            '회수금': 'sum', 
            '보상률': 'mean'
        }).reset_index()
        claims_agg.rename(columns={'국가명': '국가'}, inplace=True)
        
        # 최종 모델링 데이터셋
        model_data = pd.merge(integrated_features, claims_agg, on='국가', how='inner')
        
        print(f"\n🎯 최종 모델링 데이터셋: {model_data.shape}")
        print(f"- 분석 대상 국가: {len(model_data)}개국")
        
        # 타겟 변수 통계
        print(f"\n💰 타겟 변수 통계:")
        print(f"- 평균 보상금: {model_data['보상금'].mean():.0f}")
        print(f"- 최대 보상금: {model_data['보상금'].max():.0f}")
        print(f"- 평균 보상률: {model_data['보상률'].mean():.4f}")
        
    else:
        print("❌ 상호작용 피처 생성 실패")
else:
    print("❌ 기본 피처가 생성되지 않아 상호작용 피처를 생성할 수 없습니다")


In [ ]:
# 2-1. 앙상블 모델 학습
print("🚀 앙상블 모델 구축 시작")

if 'model_data' in locals() and not model_data.empty:
    # 피처와 타겟 분리
    feature_cols = [col for col in model_data.columns 
                   if col not in ['국가', '보상금', '회수금', '보상률']]
    
    X = model_data[feature_cols]
    y = model_data['보상금']  # 보상금을 타겟으로 설정
    
    print(f"📊 모델 학습 데이터:")
    print(f"- 피처 수: {len(feature_cols)}")
    print(f"- 샘플 수: {len(X)}")
    print(f"- 타겟: 보상금")
    
    # 앙상블 모델 구축
    ensemble_model, performance = build_ensemble_model(X, y)
    
    if ensemble_model is not None:
        print(f"\n✅ 앙상블 모델 학습 완료!")
        print(f"\n📈 모델 성능:")
        print(f"- MAE (평균절대오차): {performance['mae']:.2f}")
        print(f"- MSE (평균제곱오차): {performance['mse']:.2e}")
        print(f"- R² 점수: {performance['r2']:.4f}")
        print(f"- 훈련 데이터 크기: {performance['train_size']}")
        print(f"- 테스트 데이터 크기: {performance['test_size']}")
        
        # 피처 중요도 분석
        feature_importance = feature_importance_analysis(ensemble_model, feature_cols)
        
        if not feature_importance.empty:
            print(f"\n🔍 상위 10개 중요 피처:")
            top_features = feature_importance.head(10)
            for idx, row in top_features.iterrows():
                print(f"- {row['feature']}: {row['importance']:.4f}")
        
    else:
        print("❌ 앙상블 모델 구축 실패")
else:
    print("❌ 모델링 데이터가 준비되지 않았습니다")


In [ ]:
# 3-1. 시나리오 정의 및 예측
print("🔮 시나리오 기반 예측 수행")

if 'ensemble_model' in locals() and ensemble_model is not None:
    # 시나리오 정의
    scenarios = {
        'optimistic': '낙관적 시나리오: 수출 10% 증가, 위험지수 10% 감소',
        'realistic': '현실적 시나리오: 현재 트렌드 유지',
        'pessimistic': '비관적 시나리오: 수출 20% 감소, 위험지수 20% 증가'
    }
    
    # 시나리오별 예측
    scenario_results = predict_scenarios(ensemble_model, model_data[feature_cols], scenarios)
    
    if scenario_results:
        print(f"\n📊 시나리오별 예측 결과:")
        
        for scenario, result in scenario_results.items():
            print(f"\n🎯 {scenario.upper()}:")
            print(f"   {result['description']}")
            print(f"   - 평균 예상 보상금: {result['mean_prediction']:,.0f}")
            print(f"   - 예측 범위: {result['min_prediction']:,.0f} ~ {result['max_prediction']:,.0f}")
            print(f"   - 표준편차: {result['std_prediction']:,.0f}")
        
        # 시나리오 간 비교
        base_prediction = scenario_results['realistic']['mean_prediction']
        optimistic_change = (scenario_results['optimistic']['mean_prediction'] - base_prediction) / base_prediction * 100
        pessimistic_change = (scenario_results['pessimistic']['mean_prediction'] - base_prediction) / base_prediction * 100
        
        print(f"\n📈 현실적 시나리오 대비 변화율:")
        print(f"- 낙관적 시나리오: {optimistic_change:+.1f}%")
        print(f"- 비관적 시나리오: {pessimistic_change:+.1f}%")
        
    else:
        print("❌ 시나리오 예측 실패")
else:
    print("❌ 학습된 모델이 없어 시나리오 예측을 수행할 수 없습니다")


In [ ]:
# 4-1. 피처 중요도 시각화
if 'feature_importance' in locals() and not feature_importance.empty:
    plt.figure(figsize=(12, 8))
    
    # 상위 15개 피처
    top_15_features = feature_importance.head(15)
    
    plt.barh(range(len(top_15_features)), top_15_features['importance'], 
             color='skyblue', alpha=0.8)
    plt.yticks(range(len(top_15_features)), top_15_features['feature'])
    plt.xlabel('Feature Importance')
    plt.title('🔍 상위 15개 피처 중요도', fontsize=16, pad=20)
    plt.grid(axis='x', alpha=0.3)
    
    # 중요도 값 표시
    for i, v in enumerate(top_15_features['importance']):
        plt.text(v + 0.001, i, f'{v:.3f}', va='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    print("📊 피처 중요도 분석 결과:")
    print("- 수출 관련 피처와 위험지수 피처가 어떻게 기여하는지 확인")
    print("- 시차 피처들의 상대적 중요도 비교 가능")

# 4-2. 시나리오별 예측 결과 시각화
if 'scenario_results' in locals() and scenario_results:
    plt.figure(figsize=(10, 6))
    
    scenarios_list = list(scenario_results.keys())
    means = [scenario_results[s]['mean_prediction'] for s in scenarios_list]
    stds = [scenario_results[s]['std_prediction'] for s in scenarios_list]
    
    colors = ['green', 'blue', 'red']
    bars = plt.bar(scenarios_list, means, yerr=stds, capsize=5, 
                   color=colors, alpha=0.7)
    
    plt.title('🔮 시나리오별 예상 보상금', fontsize=16, pad=20)
    plt.ylabel('예상 보상금')
    plt.xlabel('시나리오')
    
    # 값 표시
    for bar, mean in zip(bars, means):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(stds)*0.1,
                f'{mean:,.0f}', ha='center', va='bottom', fontsize=12, fontweight='bold')
    
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("📈 시나리오 분석 해석:")
    print("- 낙관적 시나리오: 리스크 감소 효과")
    print("- 비관적 시나리오: 리스크 증가 영향")
    print("- 오차막대: 예측 불확실성 표시")


In [ ]:
# 5-1. 모델링 결과 저장
print("💾 결과 저장 중...")

try:
    # 통합 피처 데이터 저장
    if 'model_data' in locals():
        model_data.to_csv('../output/integrated_model_data.csv', index=False, encoding='cp949')
        print("✅ 통합 모델 데이터 저장 완료")
    
    # 피처 중요도 저장
    if 'feature_importance' in locals() and not feature_importance.empty:
        feature_importance.to_csv('../output/feature_importance.csv', index=False, encoding='cp949')
        print("✅ 피처 중요도 저장 완료")
    
    # 시나리오 예측 결과 저장
    if 'scenario_results' in locals() and scenario_results:
        scenario_df = pd.DataFrame([
            {
                'scenario': scenario,
                'description': result['description'],
                'mean_prediction': result['mean_prediction'],
                'std_prediction': result['std_prediction'],
                'min_prediction': result['min_prediction'],
                'max_prediction': result['max_prediction']
            }
            for scenario, result in scenario_results.items()
        ])
        scenario_df.to_csv('../output/scenario_predictions.csv', index=False, encoding='cp949')
        print("✅ 시나리오 예측 결과 저장 완료")
    
    # 모델 성능 지표 저장
    if 'performance' in locals():
        performance_df = pd.DataFrame([performance])
        performance_df.to_csv('../output/model_performance.csv', index=False, encoding='cp949')
        print("✅ 모델 성능 지표 저장 완료")
    
    print(f"\n🎯 통합 모델링 분석 완료!")
    print(f"📁 저장된 파일들:")
    print(f"- integrated_model_data.csv: 통합 모델링 데이터")
    print(f"- feature_importance.csv: 피처 중요도")
    print(f"- scenario_predictions.csv: 시나리오 예측")
    print(f"- model_performance.csv: 모델 성능")

except Exception as e:
    print(f"❌ 저장 중 오류 발생: {e}")

print(f"\n🔥 다음 단계: 05_result_visualization.ipynb로 최종 결과 시각화!")
